In [ ]:
# NER train and test file files directory
data_dir = 'drive/My Drive/Research/TurkishNER/Data/NER_TURKISH_SPLIT/'

In [ ]:
import pandas as pd
# load the training data
train_df = pd.read_csv(data_dir+"train.csv", encoding='latin5',names=['sentence_id', 'words', 'labels'])
print(train_df['sentence_id'].unique().shape)
train_df.tail(5)

(22050,)


,sentence_id,words,labels
393493,22049,her,O
393494,22049,toplumsal,O
393495,22049,sorun,O
393496,22049,politize,O
393497,22049,olur,O


In [ ]:
# load the testing data
test_df = pd.read_csv(data_dir+"test.csv", encoding='latin5',names=['sentence_id', 'words', 'labels'])
print(test_df['sentence_id'].unique().shape)
test_df.head(10)

(5513,)


,sentence_id,words,labels
0,0,ARİF,B_PERSON
1,0,:,I_PERSON
2,0,Sonradan,O
3,0,girdi,O
4,0,",",O
5,0,kendini,O
6,0,gösteremedi,O
7,1,Mahsun,B_PERSON
8,1,Kırmızıgül,I_PERSON
9,1,",",O


In [ ]:
! pip install transformers

     |████████████████████████████████| 778kB 3.4MB/s 
     |████████████████████████████████| 890kB 15.5MB/s 
     |████████████████████████████████| 1.1MB 16.8MB/s 
     |████████████████████████████████| 3.0MB 33.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=9f4d19e54ba62370a8acfc027d5f23b1740330b728021d838aa930e0790ed451
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!wget https://raw.githubusercontent.com/huggingface/transformers/master/examples/token-classification/utils_ner.py

--2020-08-15 20:11:08--  https://raw.githubusercontent.com/huggingface/transformers/master/examples/token-classification/utils_ner.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15615 (15K) [text/plain]
Saving to: ‘utils_ner.py’

utils_ner.py        100%[===================>]  15.25K  --.-KB/s    in 0.007s  

2020-08-15 20:11:08 (2.12 MB/s) - ‘utils_ner.py’ saved [15615/15615]



In [ ]:
! pip install seqeval

  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=731304557bdf0a2604cace89e547eb2457ddbf2011f7ed4695e60c2c4cc73e89
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
Successfully built seqeval


In [ ]:
model = AutoModelForTokenClassification.from_pretrained('drive/My Drive/Research/TurkishNER/Code/')

In [ ]:
import numpy as np
from seqeval.metrics import f1_score, precision_score, recall_score
from torch import nn

from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    EvalPrediction,
    HfArgumentParser,
    Trainer,
    TrainingArguments,
    set_seed,
)
from utils_ner import TokenClassificationDataset, Split

In [ ]:
AutoModelForTokenClassification

In [ ]:
train_df['labels'].unique()

array(['O', 'B_PERSON', 'I_PERSON', 'B_LOCATION', 'I_LOCATION',
       'B_ORGANIZATION', 'I_ORGANIZATION'], dtype=object)

In [ ]:
# Prepare CONLL-2003 task
labels = train_df['labels'].unique()
label_map =  {i: label for i, label in enumerate(labels)}
num_labels = len(labels)

In [ ]:
label_map

{0: 'O',
 1: 'B_PERSON',
 2: 'I_PERSON',
 3: 'B_LOCATION',
 4: 'I_LOCATION',
 5: 'B_ORGANIZATION',
 6: 'I_ORGANIZATION'}

In [ ]:
model_args = dict()
# Path to pretrained model or model identifier from huggingface.co/models
model_args['model_name_or_path'] = 'dbmdz/bert-base-turkish-cased' 

# Where do you want to store the pretrained models downloaded from s3
model_args['cache_dir'] = "BERTurk_cache/"

# we skip basic white-space tokenization by passing do_basic_tokenize = False to the tokenizer
model_args['do_basic_tokenize'] = False


In [ ]:
model_args['cache_dir']

'BERTurk_cache/'

In [ ]:
! mkdir BERTurk_cache/

In [ ]:
config = AutoConfig.from_pretrained(
    model_args['model_name_or_path'],
    num_labels=num_labels,
    id2label=label_map,
    label2id={label: i for i, label in enumerate(labels)},
    cache_dir=model_args['cache_dir']
)

# since we have our sentences already tokenized and labeled 
# retokenization by BERT_Tokenizer can cause label alignment issues
# we skip basic white-space tokenization by passing do_basic_tokenize = False to the tokenizer
# our tokenizer will only perform WordPiece tokenization

tokenizer = AutoTokenizer.from_pretrained(
    model_args['model_name_or_path'],
    cache_dir=model_args['cache_dir'],
    do_basic_tokenize = model_args['do_basic_tokenize']
)

model = AutoModelForTokenClassification.from_pretrained(
    model_args['model_name_or_path'],
    config=config,
    cache_dir=model_args['cache_dir']
)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initiali

In [ ]:
import os

def input_file_generator(df,filename,directory):
  os.makedirs(directory, exist_ok=True) 
  full_filename = os.path.join(directory, filename)
  
  current_sentence = 0
  with open(full_filename,'w') as f:
    for index, row in df.iterrows():
      if(current_sentence != row['sentence_id']):
        current_sentence += 1
        f.write('\n')
      line = "{} {}\n".format(row['words'],row['labels'])
      f.write(line)

In [ ]:
data_dir = 'data/'
input_file_generator(train_df,'train.txt',data_dir)
input_file_generator(test_df,'test.txt',data_dir)


In [ ]:
data_args = dict()
data_args['data_dir'] = data_dir 

# "The maximum total input sequence length after tokenization. Sequences longer "
# "than this will be truncated, sequences shorter will be padded."
data_args['max_seq_length'] = 512 

# Overwrite the cached training and evaluation sets
# this means the model does not have to tokenize/preprocess and cache the data each time it's called
data_args['overwrite_cache'] = False


In [ ]:
train_dataset = TokenClassificationDataset(
  data_dir=data_args['data_dir'],
  tokenizer=tokenizer,
  labels=labels,
  model_type=config.model_type,
  max_seq_length=data_args['max_seq_length'],
  overwrite_cache=data_args['overwrite_cache'],
  mode=Split.train)


In [ ]:
import json

training_args_dict = {
    'output_dir' : "model_output/",
    'num_train_epochs' : 3,
}

with open('training_args.json', 'w') as fp:
    json.dump(training_args_dict, fp)

In [ ]:
parser = HfArgumentParser(TrainingArguments)
training_args = parser.parse_json_file(json_file="training_args.json")[0] # this function returns a tuple and we only passed one arguement type "TrainingArguments"

In [ ]:
# Initialize our Trainer
trainer = Trainer(
  model=model,
  args=training_args,
  train_dataset=train_dataset,
  # eval_dataset=eval_dataset,
  # compute_metrics=compute_metrics,
)

In [ ]:
trainer.train(
    model_path=model_args['model_name_or_path']
)
trainer.save_model()

{"loss": 0.09444810306280851, "learning_rate": 4.697739088381091e-05, "epoch": 0.18135654697134568, "step": 500}


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:200: UserWarning: Please also save or load the state of the optimzer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


{"loss": 0.044720242098439486, "learning_rate": 4.395478176762181e-05, "epoch": 0.36271309394269136, "step": 1000}
{"loss": 0.04251493334735278, "learning_rate": 4.093217265143272e-05, "epoch": 0.544069640914037, "step": 1500}
{"loss": 0.036289230792230225, "learning_rate": 3.790956353524362e-05, "epoch": 0.7254261878853827, "step": 2000}
{"loss": 0.037019078376004474, "learning_rate": 3.488695441905453e-05, "epoch": 0.9067827348567283, "step": 2500}



{"loss": 0.025495528364146595, "learning_rate": 3.1864345302865435e-05, "epoch": 1.088139281828074, "step": 3000}
{"loss": 0.02107745421663276, "learning_rate": 2.8841736186676338e-05, "epoch": 1.2694958287994196, "step": 3500}
{"loss": 0.016020429265430722, "learning_rate": 2.5819127070487248e-05, "epoch": 1.4508523757707654, "step": 4000}
{"loss": 0.02045254911694792, "learning_rate": 2.279651795429815e-05, "epoch": 1.632208922742111, "step": 4500}
{"loss": 0.01829877659108024, "learning_rate": 1.9773908838109058e-05, "epoch": 1.8135654697134567, "step": 5000}
{"loss": 0.01646946950443089, "learning_rate": 1.675129972191996e-05, "epoch": 1.9949220166848023, "step": 5500}



{"loss": 0.011524935193127021, "learning_rate": 1.3728690605730868e-05, "epoch": 2.176278563656148, "step": 6000}
{"loss": 0.01044468641788626, "learning_rate": 1.0706081489541774e-05, "epoch": 2.3576351106274935, "step": 6500}
{"loss": 0.00935216430303626, "learning_rate": 7.683472373352677e-06, "epoch": 2.538991657598839, "step": 7000}
{"loss": 0.009683230199909304, "learning_rate": 4.660863257163584e-06, "epoch": 2.7203482045701852, "step": 7500}
{"loss": 0.010391792787781014, "learning_rate": 1.6382541409744893e-06, "epoch": 2.901704751541531, "step": 8000}




In [ ]:
test_dataset = NerDataset(
  data_dir=data_args['data_dir'],
  tokenizer=tokenizer,
  labels=labels,
  model_type=config.model_type,
  max_seq_length=data_args['max_seq_length'],
  overwrite_cache=data_args['overwrite_cache'],
  mode=Split.test)

In [ ]:
# last layer output/activation has the shape of (batch_size, seq_len,num_of_labels)
output, label_ids, metrics = trainer.predict(test_dataset)
preds = np.argmax(output, axis=2)
batch_size, seq_len = preds.shape
preds_list = [[] for _ in range(batch_size)]
for i in range(batch_size):
  for j in range(seq_len):
    # ignore pad_tokens
    if label_ids[i, j] != nn.CrossEntropyLoss().ignore_index:
      preds_list[i].append(label_map[preds[i][j]])



In [ ]:
def sentences_combiner(df):
  # 'words' and 'labels' are the column names in the CSV file
  tupple_function = lambda x: [(w, t) for w, t in zip(x["words"].values.tolist(),
                                                      x["labels"].values.tolist())]
  grouped = df.groupby("sentence_id").apply(tupple_function)
  return [s for s in grouped]

In [ ]:
testing_sentences = sentences_combiner(test_df)
test_labels = [[w[1] for w in s] for s in testing_sentences]
test_tokens = [[w[0] for w in s] for s in testing_sentences]

# reconstruct full sentences from lists of (token,label) tuples
test_reconstructed = [" ".join([w[0] for w in s] ) for s in testing_sentences]

In [ ]:
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
print("F1-score: {:.1%}".format(f1_score(test_labels, preds_list)))

F1-score: 84.6%


# handeling long sequences for BERT

In [ ]:
# iterate over all predictions and check which ones were not fully processed (exceeded MAX_LENGTH)
to_crop = []
for i,(x,y) in enumerate(zip(test_labels,preds_list)):
  if(len(x)!=len(y)):
    to_crop.append(i)

In [ ]:
# this function takes a list of items/tokens and creates overlapping splits of a maximum size
# we will need the overlap to provide context for both of the splitted sequences
# example sequence where this integer represent words: "[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]"
# Instead of cutting this sentence into two halves : 1-"[0, 1, 2, 3, 4, 5, 6, 7]" 2-"[8, 9, 10, 11, 12, 13, 14]"
# we can do create overlapping splits like the following: 
# [0, 1, 2, 3, 4, 5, 6]
# [5, 6, 7, 8, 9, 10, 11]
# [10, 11, 12, 13, 14]
# where 4, 5, 6 are repeated in the end and begining of seq1 and seq2
# 8, 9, 10 are repeated in the end and begining of seq2 and seq3
# this provides a better split to the sentence
def gen_split_overlap(seq, size, overlap):        
    if size < 1 or overlap < 0:
        raise ValueError('size must be >= 1 and overlap >= 0')

    for i in range(0, len(seq) - overlap, size - overlap):            
        yield seq[i:i + size]
        
# iterate over all predictions and check which ones were not fully processed (exceeded MAX_LENGTH)
long_sequences = []
for i,(x,y) in enumerate(zip(preds,real_labels)):
  if(len(x)!=len(y)):
    long_sequences.append(i)
    
#create overlapping passages for the special examples
special_tuples = []
indecies = []
for ind in long_sequences:
  ovsentence = list(gen_split_overlap(reconstructed[ind].split(), 100, 30))
  ovlabels = list(gen_split_overlap(real_labels[ind], 100, 30))
  
  for i in range(len(ovsentence)):
    # to make reconsturciton easier, the new sentence id is: oldSetenceID_OverlapPassageID
    index_label = "{}_{}".format(ind,i) 
    indecies.append(index_label)
    for token,label in zip(ovsentence[i],ovlabels[i]):
      special_tuples.append([index_label,token,label])

# previous steps can be repeated for processing the dataframe so we  create a dataframe that contains the spcial examples 
special_df = pd.DataFrame(special_tuples, columns=['sentence_id', 'words', 'labels'])

# this will replace the old test.txt
input_file_generator(special_df,'test.txt',data_dir)

# same prediction code from earlier
output, label_ids, metrics = trainer.predict(test_dataset)
preds = np.argmax(output, axis=2)
batch_size, seq_len = preds.shape
preds_list = [[] for _ in range(batch_size)]
for i in range(batch_size):
  for j in range(seq_len):
    # ignore pad_tokens
    if label_ids[i, j] != nn.CrossEntropyLoss().ignore_index:
      preds_list[i].append(label_map[preds[i][j]])


# reconstructing

In [ ]:
# reconstruct by getting the first overlapped/2 items from the first part and second overlapped/2 items from second part
def reconstruct_overlapped(lst,overlap):
  final_lst = []
  overlap_by2= int(overlap/2)
  final_lst += lst[0][:-overlap_by2]
  for i in range(len(lst)-2):
    final_lst += lst[i+1][overlap_by2:-overlap_by2]
  final_lst +=lst[-1][overlap_by2:]
  return final_lst


# combine overlaps of one example into a list
# shape: (n_examples, different_n_of_overlaps_for_each_example)
current_ind = indecies[0][:-2] #only works if the last part is a single digit (no more than 10 overlapping passages for a single training sentence)
ovr_preds = []
single_ovr_pred =[]
for lst_preds,index in zip(preds_list,indecies):
  # done with the overlapses of one sentence and checking a new one
  if(current_ind != index[:-2]): #only works if the last part is a single digit
    ovr_preds.append(single_ovr_pred)
    single_ovr_pred =[]
    current_ind = index[:-2]
  single_ovr_pred.append(lst_preds)
ovr_preds.append(single_ovr_pred)

# reconstruct the original examples 
special_preds = []
for lst in ovr_preds:
  special_preds.append(reconstruct_overlapped(lst,overlap=30))

# replace the old special examples predictions (not fully processed ones) with the new ones (after performing overlapping) 
# in the original predictions list (preds_list) so we can calculate a score for all examples
# replace the prediction of the special one and calc the total score
for i in range(len(special_preds)):
  preds_list[long_sequences[i]] = special_preds[i]